In [ ]:
import torch
from darknet53_model import Darknet53


model = Darknet53(num_classes=3)

C:\Users\user\AppData\Local\Temp\ipykernel_19032\1700460103.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location="cu

<All keys matched successfully>

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import mlflow
import mlflow.pytorch

# # ───────────────────────────────────────────────────────────
# # ✅ 로그 함수 정의
# def log_hyperparameters(params: dict):
#     for k, v in params.items():
#         mlflow.log_param(k, v)

# def log_train_metrics(loss: float, acc: float, epoch: int, prefix="train"):
#     mlflow.log_metric(f"{prefix}_loss", loss, step=epoch)
#     mlflow.log_metric(f"{prefix}_accuracy", acc, step=epoch)

# def log_test_metrics(loss: float, acc: float):
#     mlflow.log_metric("test_loss", loss)
#     mlflow.log_metric("test_accuracy", acc)

# ───────────────────────────────────────────────────────────
# ✅ 기본 설정
num_classes = 3
batch_size = 32
num_epochs = 30
lr = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("사용 중인 디바이스:", device)
print("CUDA 버전:", torch.version.cuda)
print("PyTorch 버전:", torch.__version__)
if torch.cuda.is_available():
    print("GPU 이름:", torch.cuda.get_device_name(0))
    print("GPU 총 메모리:", round(torch.cuda.get_device_properties(0).total_memory / 1024**3, 2), "GB")

# ───────────────────────────────────────────────────────────
# ✅ 모델 정의 (Darknet53 클래스 필요)
model = Darknet53(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# ───────────────────────────────────────────────────────────
# ✅ 데이터 전처리 및 로더
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

base_path = "dataset/D-Fire-split-grayscale"
train_dataset = datasets.ImageFolder(os.path.join(base_path, "train"), transform=transform)
val_dataset   = datasets.ImageFolder(os.path.join(base_path, "val"), transform=transform)
test_dataset  = datasets.ImageFolder(os.path.join(base_path, "test"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=64, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=64, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=64, pin_memory=True)

# ───────────────────────────────────────────────────────────
# ✅ MLflow 설정
# mlflow.set_tracking_uri("http://15.164.161.30:5000")
# mlflow.set_experiment("Classification-Darknet53")

# with mlflow.start_run(run_name="lr0.001_bs32"):
#     log_hyperparameters({
#         "lr": lr,
#         "batch_size": batch_size,
#         "epochs": num_epochs,
#         "model": "Darknet53"
#     })

for epoch in range(num_epochs):
    # ── Training ─────────────────────────────────────
    model.train()
    train_loss, train_correct, total = 0.0, 0, 0
    pbar = tqdm(train_loader, desc=f"[Train {epoch+1}/{num_epochs}]", leave=False)
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

        batch_acc = (predicted == labels).float().mean().item() * 100
        pbar.set_postfix({"Loss": f"{loss.item():.4f}", "Acc": f"{batch_acc:.2f}%"})

    train_loss /= total
    train_acc = train_correct / total * 100
    # log_train_metrics(train_loss, train_acc, epoch, prefix="train")

    # ── Validation ───────────────────────────────────
    model.eval()
    val_loss, val_correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss /= total
    val_acc = val_correct / total * 100
    # log_train_metrics(val_loss, val_acc, epoch, prefix="val")

    print(f"[Epoch {epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f} | Acc: {train_acc:.2f}% | "
            f"Val Loss: {val_loss:.4f} | Acc: {val_acc:.2f}%")

# ── Test Evaluation ───────────────────────────────────
model.eval()
test_loss, test_correct, total = 0.0, 0, 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        test_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_loss /= total
test_acc = test_correct / total * 100
# log_test_metrics(test_loss, test_acc)

print(f"[Test] Loss: {test_loss:.4f} | Accuracy: {test_acc:.2f}%")

# # ── 모델 저장
# mlflow.pytorch.log_model(model, "model")

# 로컬 저장 추가
os.makedirs("checkpoints", exist_ok=True)
torch.save(model.state_dict(), "checkpoints/darknet53_final.pth")
print("✅ 모델이 checkpoints/darknet53_final.pth 에 저장되었습니다.")

In [ ]:
import torch
from darknet53_model import Darknet53  # 반드시 동일한 모델 정의 필요

# 설정
num_classes = 3
model_path = "checkpoints/darknet53_final.pth"

# 모델 정의 및 로드
model = Darknet53(num_classes=num_classes)
model.load_state_dict(torch.load(model_path, map_location="cuda"))  # GPU 사용 시 map_location="cuda"
model.eval()

print("✅ 모델이 성공적으로 로드되었습니다.")

model.eval()

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])


image_dir = "dataset/for_test"
output_dir = "dataset/test_labeled"
os.makedirs(output_dir, exist_ok=True)

# 이미지 파일만 필터링 (jpg, jpeg, png 등)
image_paths = [
    os.path.join(image_dir, fname)
    for fname in os.listdir(image_dir)
    if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
]

class_names = ["smoke", "fire", "normal"]  # 실제 학습 시 클래스 순서대로 맞춰야 함


for img_path in image_paths:
    image = Image.open(img_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)  # (1, 3, 224, 224)

    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted = torch.max(outputs, 1)
        predicted_class = class_names[predicted.item()]

        # 🔽 파일명 조합
        base_name, ext = os.path.splitext(os.path.basename(img_path))
        new_filename = f"{base_name}_{predicted_class}{ext}"
        save_path = os.path.join(output_dir, new_filename)

        # 🔽 이미지 저장
        image.save(save_path)

    print(f"📷 {os.path.basename(img_path)} → 예측 결과: {predicted_class}")
